In [ ]:
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt

# Start a Spark Session
spark = SparkSession.builder \
    .appName("SmartSales") \
    .config("spark.jars", "/Users/silvertiger/Projects/smart-store-drew-schaffner/lib/sqlite-jdbc-3.49.1.0.jar") \
    .config("spark.drive.extraClassPath", "/Users/silvertiger/Projects/smart-store-drew-schaffner/lib/sqlite-jdbc-3.49.1.0.jar") \
    .getOrCreate()

# Load sale table
df_sale = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/Users/silvertiger/Projects/smart-store-drew-schaffner/data/dw/smart_sales.db") \
    .option("dbtable", "sale") \
    .option("driver", "org.sqlite.JDBC") \
    .load()

# Load customer table
df_customer = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/Users/silvertiger/Projects/smart-store-drew-schaffner/data/dw/smart_sales.db") \
    .option("dbtable", "customer") \
    .option("driver", "org.sqlite.JDBC") \
    .load()

# Load product table
df_product = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/Users/silvertiger/Projects/smart-store-drew-schaffner/data/dw/smart_sales.db") \
    .option("dbtable", "product") \
    .option("driver", "org.sqlite.JDBC") \
    .load()

# Un# to show the dataframes
df_customer.show()
df_sale.show()
df_product.show()


# Register DataFrames as temporary views (if not already done)
df_sale.createOrReplaceTempView("sale")
df_customer.createOrReplaceTempView("customer")
df_product.createOrReplaceTempView("product")

# print(spark.catalog.listTables())

# Write query using Spark SQL
df_top_customers = spark.sql("""
SELECT c.name, SUM(s.sale_amount) AS total_spent
FROM sale s
JOIN customer c ON s.customer_id = c.customer_id
GROUP BY c.name
ORDER BY total_spent DESC
""")

df_sales_trends = spark.sql("""
SELECT
    DATE_TRUNC('month', to_timestamp(s.sale_date, 'M/d/yy')) AS sale_month,
    SUM(s.sale_amount) AS monthly_sales,
    p.category AS product_category
FROM
    sale s
JOIN
    product p ON s.product_id = p.product_id
GROUP BY
    sale_month, p.category
ORDER BY
    sale_month
""")

# Show spark results
# df_top_customers.show()
df_sales_trends.show()

# Convert to Pandas DataFrame for visualization
df_top_customers_pd = df_top_customers.toPandas()

In [ ]:
# This code does nothing for me but hypothetically, if my data contained a sale date that took place prior to 2023-01-01, I could filter it out. 
df_filtered = df_sale.filter(df_sale.sale_date >= "2023-01-01")

df_sale.groupby("product_id").sum("sale_amount").show()

df_sale.groupby("campaign_id").sum("sale_amount").show()

sns.barplot(data=df_top_customers_pd, x="name", y="total_spent")
plt.xticks(rotation=45)
plt.show()

sns.lineplot(data = df_sales_trends.toPandas(), x="sale_month", y="monthly_sales", hue="product_category")
plt.xticks(rotation=45)
plt.show()
